https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/120.0.6099.109/win64/chromedriver-win64.zip

In [1]:
#imports here
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.service import Service
import time

In [2]:
#code by pythonjar, not me
service = Service(executable_path='chromedriver.exe')
options = webdriver.ChromeOptions()
options.add_argument('--disable-notifications')

In [481]:
from dotenv import load_dotenv, dotenv_values

if not load_dotenv():
    raise Exception(".env not found.")

driver = webdriver.Chrome(service=service, options=options)
driver.get("https://mbasic.facebook.com/login.php")

config = dotenv_values(".env")

username = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='email']"))
)
password = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='pass']"))
)

# enter username and password
username.clear()
username.send_keys(config["EMAIL"])
password.clear()
password.send_keys(config["PASS"])
WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'input[type="submit"]'))
).click()
time.sleep(1)
WebDriverWait(driver, 2).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, 'div a[class="bo bp bq br bt"]'))
).click()
time.sleep(1)

In [496]:
import re

def replies_scraping(box_comment):
    try:
        replies_url = (
            box_comment
            .find_element(By.CSS_SELECTOR, "div > div:nth-child(5) > div[id] a[href]")
            .get_attribute("href")
        )
        driver.execute_script(f"window.open('{replies_url}', '_blank')")
        driver.switch_to.window(driver.window_handles[2])
    except:
        return None

    replie_id = re.search(r"ctoken=(\d+_\d+)", driver.current_url).group(1)
    replies = []

    while True:
        box_replies = driver.find_elements(
            By.CSS_SELECTOR,
            'div[id="root"] > div[class] > div[id] + div > div:has(div)',
        )
        for box in box_replies:
            replie = dict()
            replie["replie_by"] = box.find_element(By.CSS_SELECTOR, "div > h3").text
            replie["replie"] = box.find_element(
                By.CSS_SELECTOR, "div > div:nth-child(2)"
            ).text
            replies.append(replie)
        try:
            prev_comment_btn = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable(
                    (
                        By.CSS_SELECTOR,
                        f'div[id="root"] > div[class] > div[id] + div > div[id="comment_replies_more_1:{replie_id}"] > a',
                    )
                )
            )
            prev_comment_btn.click()
        except:
            print("No additional replies.")
            break
    driver.close()
    time.sleep(.5)
    driver.switch_to.window(driver.window_handles[1])
    return replies

In [501]:
import re


def comments_scraping():
    post_id = re.search(r"/permalink/(\d+)", driver.current_url).group(1)
    comments = []

    while True:
        box_comments = driver.find_elements(
            By.CSS_SELECTOR,
            "#m_story_permalink_view > div.bi + div > div > div:not([id]) > div:has(div)",
        )

        for box_comment in box_comments:
            comment = dict()
            comment["comment_by"] = box_comment.find_element(
                By.CSS_SELECTOR, "div > h3"
            ).text
            comment["comment"] = box_comment.find_elements(
                By.CSS_SELECTOR, "div > div:nth-child(2)"
            )[0].text
            driver.switch_to.window(driver.window_handles[1])

            replies = replies_scraping(box_comment)
            if replies:
                # print(replies)
                comment["replies"] = replies

            comments.append(comment)

        try:
            prev_comment_btn = WebDriverWait(driver, 2).until(
                EC.element_to_be_clickable(
                    (
                        By.CSS_SELECTOR,
                        f'#m_story_permalink_view > div.bi + div > div > div:not([id]) > div[id="see_prev_{post_id}"] > a',
                    )
                )
            )
            prev_comment_btn.click()
        except:
            # print("No additional comments.")
            break
    print(f"Complete post id: {post_id}")
    return comments

In [497]:
import re

def post_scraping():
    post_id = re.search(r"/permalink/(\d+)", driver.current_url).group(1)
    post = dict()
    post["post_id"] = post_id
    post["topic"] = driver.find_element(
        By.CSS_SELECTOR,
        'div[id="m_story_permalink_view"] div.bi div.bj.bk div.bl > div',
    ).text
    post["topic_by"] = driver.find_element(
        By.CSS_SELECTOR, "td.t.bo header h3 span strong a"
    ).text
    post["comments"] = comments_scraping()
    return post

In [526]:
driver.switch_to.window(driver.window_handles[0])

main_group_link = "https://mbasic.facebook.com/groups/1991721561082549/"
driver.get(main_group_link)

In [ ]:
anchors = driver.find_elements(By.CSS_SELECTOR, 'div.cp.cq a')
anchors = [a.get_attribute('href') for a in anchors if (a.text == 'เรื่องราวฉบับเต็ม') or (a.text == 'Full Story')]

posts = []

for a in anchors:
    driver.execute_script(f"window.open('{a}', '_blank')")
    driver.switch_to.window(driver.window_handles[1])
    post = post_scraping()
    posts.append(post)
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

# ไม่เกี่ยว

In [525]:
import json

with open('data.json', 'a', encoding="utf-8-sig") as file:
  json.dump(posts, file, indent=2, ensure_ascii=False)

In [ ]:
import pyperclip
pyperclip.copy(anchors[0])